### Install the _Kubeflow-metadata_ library

In [ ]:
# To use the latest publish `kubeflow-metadata` library, you can run:
!pip3 install kubeflow-metadata --user
# Install other packages:
!pip3 install pandas --user
# Then restart the Notebook kernel.

In [3]:
import pandas
from kubeflow.metadata import metadata
from datetime import datetime
from uuid import uuid4

In [4]:
METADATA_STORE_HOST = "metadata-grpc-service.kubeflow" # default DNS of Kubeflow Metadata gRPC serivce.
METADATA_STORE_PORT = 8080

### Create a workspace

In [5]:
ws1 = metadata.Workspace(
    # Connect to metadata service in namespace kubeflow in k8s cluster.
    store=metadata.Store(grpc_host=METADATA_STORE_HOST, grpc_port=METADATA_STORE_PORT),
    name="vult5",
    description="a workspace for testing",
    labels={"n1": "v1"})

### Create a run in a workspace

In [6]:
r = metadata.Run(
    workspace=ws1,
    name="run-" + datetime.utcnow().isoformat("T") ,
    description="a run in ws_1",
)

### Create an execution in a run

In [7]:
exec = metadata.Execution(
    name = "execution" + datetime.utcnow().isoformat("T") ,
    workspace=ws1,
    run=r,
    description="execution example",
)
print("An execution was created with id %s" % exec.id)

An execution was created with id 3


### Log a data set

In [8]:
date_set_version = "data_set_version_" + str(uuid4())
data_set = exec.log_input(
        metadata.DataSet(
            description="an example data",
            name="mytable-dump",
            owner="owner@my-company.org",
            uri="file://path/to/dataset",
            version=date_set_version,
            query="SELECT * FROM mytable"))
print("Data set id is {0.id} with version '{0.version}'".format(data_set))

Data set id is 4 with version 'data_set_version_7b540b62-918d-4344-a8f1-fb3a2e1e2dce'


### Log a model

In [9]:
model_version = "model_version_" + str(uuid4())
model = exec.log_output(
    metadata.Model(
            name="MNIST",
            description="model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist",
            model_type="neural network",
            training_framework={
                "name": "tensorflow",
                "version": "v1.0"
            },
            hyperparameters={
                "learning_rate": 0.5,
                "layers": [10, 3, 1],
                "early_stop": True
            },
            version=model_version,
            labels={"mylabel": "l1"}))
print(model)
print("\nModel id is {0.id} and version is {0.version}".format(model))

ERROR:absl:mlmd client InternalError: mysql_query failed: errno: 2013, error: Lost connection to MySQL server during query


kubeflow.metadata.metadata.Model(workspace=None, name='MNIST', description='model to recognize handwritten digits', owner='someone@kubeflow.org', uri='gcs://my-bucket/mnist', version='model_version_d241c700-bf7a-4742-9871-2aa50c75d987', model_type='neural network', training_framework={'name': 'tensorflow', 'version': 'v1.0'}, hyperparameters={'learning_rate': 0.5, 'layers': [10, 3, 1], 'early_stop': True}, labels={'mylabel': 'l1'}, id=5, create_time='2020-09-24T11:57:26.401381Z', kwargs={})

Model id is 5 and version is model_version_d241c700-bf7a-4742-9871-2aa50c75d987


### Log the evaluation of a model

In [10]:
metrics = exec.log_output(
    metadata.Metrics(
            name="MNIST-evaluation",
            description="validating the MNIST model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist-eval.csv",
            data_set_id=str(data_set.id),
            model_id=str(model.id),
            metrics_type=metadata.Metrics.VALIDATION,
            values={"accuracy": 0.95},
            labels={"mylabel": "l1"}))
print("Metrics id is %s" % metrics.id)

Metrics id is 6


### Add Metadata for serving the model

In [11]:
serving_application = metadata.Execution(
    name="serving model",
    workspace=ws1,
    description="an execution to represent model serving component",
)
# Noticed we use model name, version, uri to uniquely identify existing model.
served_model = metadata.Model(
    name="MNIST",
    uri="gcs://my-bucket/mnist",
    version=model.version,
)
m=serving_application.log_input(served_model)
print("Found the mode with id {0.id} and version '{0.version}'.".format(m))

Found the mode with id 5 and version 'model_version_d241c700-bf7a-4742-9871-2aa50c75d987'.


### List all models in the workspace

In [12]:
pandas.DataFrame.from_dict(ws1.list(metadata.Model.ARTIFACT_TYPE_NAME))

,id,workspace,run,version,owner,description,name,model_type,create_time,uri,training_framework,hyperparameters,labels,kwargs
0,2,vult5,run-2020-08-18T07:16:05.084579,model_version_b22de54d-616c-4ab5-acd8-59e0ae32...,someone@kubeflow.org,model to recognize handwritten digits,MNIST,neural network,2020-08-18T07:22:43.408528Z,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}
1,5,vult5,run-2020-09-24T11:29:46.146791,model_version_d241c700-bf7a-4742-9871-2aa50c75...,someone@kubeflow.org,model to recognize handwritten digits,MNIST,neural network,2020-09-24T11:57:26.401381Z,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}


### Basic Lineage Tracking

In [13]:
print("Model id is %s\n" % model.id)
    
model_events = ws1.store.get_events_by_artifact_ids([model.id])

execution_ids = set(e.execution_id for e in model_events)
print("All executions related to the model are {}".format(execution_ids))
# assert execution_ids == set([serving_application.id, exec.id])

trainer_events = ws1.store.get_events_by_execution_ids([exec.id])
artifact_ids = set(e.artifact_id for e in trainer_events)
print("All artifacts related to the training event are {}".format(artifact_ids))# assert artifact_ids == set([model.id, metrics.id, data_set.id])

Model id is 5

All executions related to the model are {3, 4}
All artifacts related to the training event are {4, 5, 6}
